In [1]:
import numpy as np
import copy
import math
import random 
def turnIntoMatrix(filepath):
    file = open(filepath, "r")
    lines = file.readlines()
    file.close()
    
    ##Preparing the list of lists 
    matrix = []
    
    for line in lines:
        ##sample will store all the values found 
        ##in a line which represents a sample when the string values were turned into numbers 
        sample = []
        line = line.replace("\n", "")
        split = line.split(",")
        for value in split:
            sample.append(float(value))
        matrix.append(sample)
    return matrix 
def euclideanDistance(point1 , point2): 
    if len(point1) != len(point2):
        raise Exception("List lenght doesn't match")
    else: 
        i = 0
        squareSum= 0
        eucliDistance = 0
        while i < len(point1):
            squareSum = squareSum +pow(point1[i] - point2[i],2)
            i = i+1
        eucliDistance = pow(squareSum, 0.5)
        return eucliDistance

##Calculate manhattan distance for any two points with multivariables 
def manhattanDistance(point1, point2):
    if len(point1) != len(point2):
        raise Exception("List lenght doesn't match")
    else:
        ##Calculate 
        i = 0
        blockSum = 0
        while i < len(point1):
            blockSum = blockSum + abs(point1[i] - point2[i])
            i=i+1
        return blockSum
##A simple way to find the closest neighbours 
def getNeighbour(typeofdistance, matrix, sample, neighbours):
    distance = []
    euclidean = "euclidean"
    manhattan = "manhattan"
    if typeofdistance == euclidean:
        for row in matrix:
            euclidean = euclideanDistance(sample[:len(sample)-1], row[:len(sample)-1])
            label = row[len(row)-1]
            distance.append([euclidean, label])
    elif typeofdistance == manhattan:
        for row in matrix:
            manhattan = manhattanDistance(sample[:len(sample)-1], row[:len(row)-1])
            label = row[len(row)-1]
            distance.append([manhattan, label])
    distance.sort(key = lambda x: x[0])
    return distance[:neighbours]
def byVote(distances):
    classes = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0
    }
    for distance in distances:
        if distance[1] == 1.0:
            classes[1] += 1
        elif distance[1] == 2.0:
            classes[2] += 1
        elif distance[1] == 3.0:
            classes[3] += 1
        elif distance[1] == 4.0: 
            classes[4] += 1
        elif distance[1] == 5.0:
            classes[5] += 1
        elif distance[1] == 6.0:
            classes[6] += 1
        else: 
            raise Exception("Value not recognizable")
    votes = max(classes.values())
    winner = max(classes, key=classes.get)
    return (winner, votes)

In [2]:
matrix = turnIntoMatrix("dermatology.data.treated")
matriz = np.array(matrix)
print(matriz)

[[ 2.  2.  0. ...  0. 55.  2.]
 [ 3.  3.  3. ...  0.  8.  1.]
 [ 2.  1.  2. ...  3. 26.  3.]
 ...
 [ 3.  2.  2. ...  3. 28.  3.]
 [ 2.  1.  3. ...  3. 50.  3.]
 [ 3.  2.  2. ...  0. 35.  1.]]


In [3]:
#Criando média da matriz e subtraindo das amostras para centralização
def subtract(matrix, subarray):
    j = 0
    mat = np.copy(matrix) 
    while j < len(matrix): 
        line = mat[j]
        i = 0
        while i < 34:
            line[i] = line[i] - subarray[i]
            i = i + 1
        mat[j] = line
        j = j + 1
    return matrix
def pca(matrix, Q):
    #calculate the mean and subtract from all lines of the original matrix 
    mean = np.mean(matrix[:, :34], axis = 0)
    matriz_meaned = subtract(matrix, mean)
    #extract covariance matrix from the meaned matrix 
    cov_matrix = np.cov(matriz_meaned[:, :34], rowvar = False)
    eigen_values, eigen_vectors = np.linalg.eigh(cov_matrix)
  
    #sort the eigen_values in descending order 
    eigen_values_args_sorted = np.argsort(eigen_values)[::-1]
    eigen_values_sorted = eigen_values[eigen_values_args_sorted]
    #similarly sort the eigenvectors 
    eigen_vectors_sorted = eigen_vectors[:, eigen_values_args_sorted]
    #reduce matrix
    eigen_vector_sub = eigen_vectors_sorted[:, 0:Q]
    reduced_matrix = np.dot(eigen_vector_sub.transpose(), matriz_meaned[:,:34].transpose()).transpose()
    reduced_matrix = np.append(reduced_matrix, matriz_meaned[:,34:], axis = 1)
    #reduced_matrix = np.append(reduced_matrix, )
    return reduced_matrix, eigen_values_sorted, eigen_vector_sub

def getRandomDataSet(matrix, percentage):
    numberOfSamples = int(round(percentage*len(matrix)))
    trainingSet = random.sample(matrix, numberOfSamples)
    testSet = [elem for elem in matrix if elem not in trainingSet]
    return trainingSet, testSet

#escolha Q de 0.95
def chooseQ(matrix, percent):
    q = 1 
    while q < 34:
        reduced, eigen, _ = pca(matrix, q)
        result = sum(eigen[:q])/sum(eigen)
        print(result)
        if result >= percent:
            return q, reduced
        q = q + 1
    return q, reduced


In [4]:
##Vamos executar o teste do classificador utilizando KNN para determinar a taxa de acerto máxima 
##Também vamos observar se o Q máximo é o mesmo que refere aos 0.95 da questão anterior
#Vamos executar o mesmo teste várias vezes até achar o Q que é máximo
training, test = getRandomDataSet(matriz.tolist(), 0.8)
#média do grupo de treino
q, reduced = chooseQ(np.array(training), 0.95) #0.95 representa os 95 de variancia aproximada para o número de componentes 
print("O resulta é q: " +str(q)+ " dimensões")
print("###################Matriz reduzida#####################")
print(reduced)
mean = np.mean(training, axis=0)
#centralizando grupo de teste 
meaned_test = subtract(np.array(test), mean)

0.9015643574883447
0.938029507205683
0.9561273404981342
O resulta é q: 3 dimensões
###################Matriz reduzida#####################
[[-34.98093593  -1.07900058   0.87039233   2.        ]
 [-17.12163539   3.74586794  -4.89187451   1.        ]
 [-32.09560708   1.39299437  -2.89197037   1.        ]
 ...
 [-36.08524286   3.31539641  -3.5610461    1.        ]
 [-47.05608107   1.80658251  -1.74326947   1.        ]
 [-42.12251992  -5.06627518  -3.18257137   3.        ]]


In [5]:
def prediction(traingroup, testsample, typeofdistance, knn):
    distances = getNeighbour(typeofdistance, traingroup, testsample, knn)
    (winner, votes) = byVote(distances)
    return (winner, votes)

def testFunc(training, test):
    maxQ = 1
    maxAcertos = 0
    for i in range(1,35):
        #pca do grupo de treino
        training_reduced, eigen_value, eigen_vector = pca(np.array(training), i)
        #Transformando o grupo de teste e de treino
        reduced_test = np.dot(eigen_vector.transpose(), meaned_test[:,:34].transpose()).transpose()
        reduced_test = np.append(reduced_test, meaned_test[:,34:], axis = 1)
        numeroDeAcertos = 0
        for line in reduced_test: 
            winner, votes = prediction(training_reduced.tolist(), line.tolist(), "manhattan", 3)
            if line[len(line)-1] == winner:
                numeroDeAcertos = numeroDeAcertos + 1
        print("Dimensões " +str(i)+ " Acertos do KNN "+str(numeroDeAcertos))
        if numeroDeAcertos > maxAcertos: 
            maxQ = i 
            maxAcertos = numeroDeAcertos 
    return maxQ, maxAcertos

def testFunc2(training, test):
    maxQ = 1
    maxAcertos = 0
    for i in range(1,35):
        #pca do grupo de treino
        training_reduced, eigen_value, eigen_vector = pca(np.array(training), i)
        #Transformando o grupo de teste e de treino
        reduced_test = np.dot(eigen_vector.transpose(), meaned_test[:,:34].transpose()).transpose()
        reduced_test = np.append(reduced_test, meaned_test[:,34:], axis = 1)
        numeroDeAcertos = 0
        for line in reduced_test: 
            winner, votes = prediction(training_reduced.tolist(), line.tolist(), "manhattan", 3)
            if line[len(line)-1] == winner:
                numeroDeAcertos = numeroDeAcertos + 1
        #print("Dimensões " +str(i)+ " Acertos do KNN "+str(numeroDeAcertos))
        if numeroDeAcertos > maxAcertos: 
            maxQ = i 
            maxAcertos = numeroDeAcertos 
    return maxQ, maxAcertos
    

In [6]:
maxQ, maxAcertos = testFunc(training, test)
print("Um das máximas está na dimensão " + str(maxQ)+ " com o número de acertos de " +str(maxAcertos))

Dimensões 1 Acertos do KNN 29
Dimensões 2 Acertos do KNN 46
Dimensões 3 Acertos do KNN 48
Dimensões 4 Acertos do KNN 55
Dimensões 5 Acertos do KNN 59
Dimensões 6 Acertos do KNN 63
Dimensões 7 Acertos do KNN 64
Dimensões 8 Acertos do KNN 64
Dimensões 9 Acertos do KNN 63
Dimensões 10 Acertos do KNN 62
Dimensões 11 Acertos do KNN 66
Dimensões 12 Acertos do KNN 65
Dimensões 13 Acertos do KNN 63
Dimensões 14 Acertos do KNN 64
Dimensões 15 Acertos do KNN 63
Dimensões 16 Acertos do KNN 64
Dimensões 17 Acertos do KNN 63
Dimensões 18 Acertos do KNN 63
Dimensões 19 Acertos do KNN 63
Dimensões 20 Acertos do KNN 62
Dimensões 21 Acertos do KNN 63
Dimensões 22 Acertos do KNN 63
Dimensões 23 Acertos do KNN 64
Dimensões 24 Acertos do KNN 64
Dimensões 25 Acertos do KNN 64
Dimensões 26 Acertos do KNN 64
Dimensões 27 Acertos do KNN 64
Dimensões 28 Acertos do KNN 64
Dimensões 29 Acertos do KNN 64
Dimensões 30 Acertos do KNN 64
Dimensões 31 Acertos do KNN 64
Dimensões 32 Acertos do KNN 64
Dimensões 33 Acer

In [7]:
#Assim observamos se a dimensão que obteve o máximo de acertos corresponde a variância de 0.95
#O resultado é que, como o grupo de treino e teste são randomicos, a variância provavelmente não será de 0.95, provavelmente
#será maior
_, eigen, _ = pca(np.array(training), maxQ)
result = sum(eigen[:maxQ])/sum(eigen)
print("O resultado da variância explicada é " + str(result))


O resultado da variância explicada é 0.983541016194749


In [8]:
#Vamos executar a classificação no grupo de teste e treino sem executar a redução de dimensões com o método PCA
#O teste será realizado 100 vezes depois tiraremos a média de acertos e o desvio padrão, o teste está sendo feito considerando 
#todas as classes, já que o documento do trabalho não especifica se a taxa de acertos era por classe ou total
#Queremos argumentar que o método utilizado para a resolução da questão 1.1 
#também serve para extrair os componentes principais mais importantes onde a taxa de acertos é máxima com dimensões mínimas
#logo repetiremos os testes com o mesmo método, considerando apenas que como o grupo de treino e teste são randomicos
#as dimensões Q podem variar 
acertosAntesDaReducao = np.zeros(100)
acertosDepoisDaReducao = np.zeros(100)
for i in range(100):
    training, test = getRandomDataSet(matriz.tolist(), 0.8)
    for line in test: 
        winner, votes = prediction(training, line, "manhattan", 3)
        if line[len(line)-1] == winner:
            acertosAntesDaReducao[i] = acertosAntesDaReducao[i] + 1
            
            
    maxQ, maxAcertos = testFunc2(training, test)
    print("############################################NOVO TESTE######################################")
    print("Teste #" +str(i+1)+ " dimensões = 34 número de acertos = " +str(acertosAntesDaReducao[i]))
    print("Teste #" +str(i+1)+ " dimensões = " +str(maxQ)+ " número de acertos = " +str(maxAcertos))
    print("############################################NOVO TESTE######################################")
    acertosDepoisDaReducao[i] = maxAcertos 



############################################NOVO TESTE######################################
Teste #1 dimensões = 34 número de acertos = 69.0
Teste #1 dimensões = 17 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #2 dimensões = 34 número de acertos = 66.0
Teste #2 dimensões = 7 número de acertos = 69
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #3 dimensões = 34 número de acertos = 66.0
Teste #3 dimensões = 9 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #4 dimensões = 34 número de acertos = 67.0
Teste #4 dimen

############################################NOVO TESTE######################################
Teste #30 dimensões = 34 número de acertos = 66.0
Teste #30 dimensões = 9 número de acertos = 69
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #31 dimensões = 34 número de acertos = 67.0
Teste #31 dimensões = 7 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #32 dimensões = 34 número de acertos = 70.0
Teste #32 dimensões = 7 número de acertos = 67
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #33 dimensões = 34 número de acertos = 67.0
Teste #3

############################################NOVO TESTE######################################
Teste #59 dimensões = 34 número de acertos = 65.0
Teste #59 dimensões = 9 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #60 dimensões = 34 número de acertos = 68.0
Teste #60 dimensões = 20 número de acertos = 69
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #61 dimensões = 34 número de acertos = 65.0
Teste #61 dimensões = 6 número de acertos = 67
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #62 dimensões = 34 número de acertos = 71.0
Teste #

############################################NOVO TESTE######################################
Teste #88 dimensões = 34 número de acertos = 70.0
Teste #88 dimensões = 8 número de acertos = 67
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #89 dimensões = 34 número de acertos = 67.0
Teste #89 dimensões = 7 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #90 dimensões = 34 número de acertos = 64.0
Teste #90 dimensões = 7 número de acertos = 68
############################################NOVO TESTE######################################
############################################NOVO TESTE######################################
Teste #91 dimensões = 34 número de acertos = 68.0
Teste #9

In [9]:
def desvioPadrao(array, media):
    soma = 0
    for i in array:
        soma = soma + (i-media)*(i-media)
    desvio = soma/len(array)
    desvio = pow(desvio, 0.5)
    return desvio
print(acertosAntesDaReducao)
mediaAntes = acertosAntesDaReducao.sum()/len(acertosAntesDaReducao)
desvioAntes = desvioPadrao(acertosAntesDaReducao, mediaAntes)
mediaDepois = acertosDepoisDaReducao.sum()/len(acertosDepoisDaReducao)
desvioDepois = desvioPadrao(acertosDepoisDaReducao, mediaAntes)
print("Média de acertos antes da redução " +str(mediaAntes) + "Desvio Padrão " + str(desvioAntes))

print("Média de acertos depois da redução " +str(mediaDepois)+ " Desvio Padrão " + str(desvioDepois))

[69. 66. 66. 67. 65. 68. 65. 68. 71. 64. 70. 66. 62. 67. 68. 71. 69. 66.
 71. 66. 69. 65. 68. 71. 66. 65. 66. 69. 69. 66. 67. 70. 67. 64. 71. 67.
 68. 67. 67. 66. 69. 71. 68. 70. 64. 67. 66. 70. 70. 65. 66. 64. 69. 69.
 70. 66. 67. 67. 65. 68. 65. 71. 69. 68. 68. 62. 69. 67. 68. 65. 67. 69.
 70. 63. 69. 69. 67. 68. 64. 71. 68. 67. 69. 68. 64. 65. 66. 70. 67. 64.
 68. 71. 64. 66. 66. 70. 67. 65. 67. 69.]
Média de acertos antes da redução 67.34Desvio Padrão 2.1918941580286244
Média de acertos depois da redução 68.22 Desvio Padrão 1.366016105322332
